In [2]:
import win32com.client
import ctypes
from pywinauto import application
import pandas as pd
from datetime import datetime
import time

# 서버 접속 확인
objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
bConnect = objCpCybos.IsConnect

def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자 권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반 권한으로 실행됨. 관리자 권한으로 실행해주세요.')
        return False

    if (bConnect == 1): #접속 - 1, 비접속 - 0
        print("정상: 서버에 접속되었습니다.")
    else:
        print("오류: 서버에 연결되지 않았습니다. ")
        return False

InitPlusCheck()

정상: 관리자 권한으로 실행된 프로세스입니다.
정상: 서버에 접속되었습니다.


In [111]:
# 오브젝트 요청
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
objRq = win32com.client.Dispatch("CpSysDib.MarketEye")

codeList = g_objCodeMgr.GetStockListByMarket(1)  # 거래소
codeList2 = g_objCodeMgr.GetStockListByMarket(2)  # 코스닥
allcodelist = codeList + codeList2 #전체 코드리스트

hugeCodeList = [g_objCodeMgr.IsBigListingStock(i) for i in allcodelist] # 상장주식수 수신시 1000단위로 받아옴

rqField = [0,4,5,6,7,17,20] #요청 필드// 0종목코드, 4현재가, 5시가, 6고가, 7저가, 17종목명, 20상장주식수
rqCodeList = []

sumcnt = 0 # 가져온 데이터 개수
df = pd.DataFrame(columns=('종목코드', '종목명', '현재가', '상장주식수', '시가총액')) 


codeindex = 0 # 코드리스트 200개씩 받아오기위한 인덱스
allcodeindex = len(allcodelist) # 전체 코드개수
while True:
    rqCodeList = []
    for i in range(200): # 일단 200개씩 추가
        if allcodeindex <= codeindex + i: # 최대 코드개수를 초과하면 멈춤
            break
        rqCodeList.append(allcodelist[codeindex + i])
    codeindex += len(rqCodeList) # 다음200개 준비

    remainCount = g_objCpStatus.GetLimitRemainCount(1)  # 1 시세 제한
    if remainCount <= 0:
        print('시세 연속 조회 제한 회피를 위해 sleep', g_objCpStatus.LimitRequestRemainTime)
        print(remainCount)
        time.sleep(g_objCpStatus.LimitRequestRemainTime / 1000)
    
    objRq.SetInputValue(0, rqField) # 받아올 데이터 지정
    objRq.SetInputValue(1, rqCodeList) # 받아올 종목 지정

    objRq.BlockRequest()

    cnt = objRq.GetHeaderValue(2)
    sumcnt += cnt
    print(cnt)

    for i in range(cnt):
        item = {}
        item['종목코드'] = objRq.GetDataValue(0, i) # 종목코드
        item['종목명'] = objRq.GetDataValue(5, i) # 종목명
        item['현재가'] = objRq.GetDataValue(1, i) # 현재가
        item['상장주식수'] = objRq.GetDataValue(6, i) # 상장주식수
        if hugeCodeList[sumcnt - cnt + i]:
            item['상장주식수'] *= 1000
        item['시가총액'] = int(item['상장주식수'] * item['현재가'] / 100000000)
        item['시가총액'] = format(item['시가총액'],',')
        df.loc[len(df)] = item

    if sumcnt >= allcodeindex:
        break


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
179


In [112]:
df

,종목코드,종목명,현재가,상장주식수,시가총액
0,A000020,동화약품,9670,27931470,"2,700"
1,A000040,KR모터스,629,96137631,604
2,A000050,경방,12300,27415270,"3,372"
3,A000060,메리츠화재,34350,120625000,"41,434"
4,A000070,삼양홀딩스,74400,8564271,"6,371"
...,...,...,...,...,...
3374,A950160,코오롱티슈진,8010,64920050,"5,200"
3375,A950170,JTC,2880,35005517,"1,008"
3376,A950190,미투젠,16200,13579892,"2,199"
3377,A950200,소마젠,5690,19133553,"1,088"
